In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/00235/household_power_consumption.zip
!unzip household_power_consumption.zip

In [11]:
# The ds (datestamp) column should be of a format expected by Pandas,
# ideally YYYY-MM-DD for a date or YYYY-MM-DD HH:MM:SS for a timestamp.

df = pd.read_csv('household_power_consumption.txt', sep=';')

C:\Users\jaros\AppData\Local\Temp\ipykernel_8040\293355836.py:4: DtypeWarning: Columns (2,3,4,5,6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('household_power_consumption.txt', sep=';')


In [12]:
df['DateTime'] = pd.to_datetime(df['Date'] + ' ' + df['Time'])

del df['Date']
del df['Time']

df.set_index("DateTime", inplace=True)

C:\Users\jaros\AppData\Local\Temp\ipykernel_8040\1650630806.py:1: UserWarning: Parsing dates in %d/%m/%Y %H:%M:%S format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['DateTime'] = pd.to_datetime(df['Date'] + ' ' + df['Time'])


In [13]:
df.isna().sum()

Global_active_power          0
Global_reactive_power        0
Voltage                      0
Global_intensity             0
Sub_metering_1               0
Sub_metering_2               0
Sub_metering_3           25979
dtype: int64

In [14]:
df.describe(include='all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
Global_active_power,2075259,6534,?,25979,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Global_reactive_power,2075259,896,0.000,472786,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Voltage,2075259,5168,?,25979,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Global_intensity,2075259,377,1.000,169406,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Sub_metering_1,2075259,153,0.000,1840611,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Sub_metering_2,2075259,145,0.000,1408274,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Sub_metering_3,2049280.0,NaN,NaN,NaN,6.458447,8.437154,0.0,0.0,1.0,17.0,31.0


In [15]:
df.isin(['?']).sum()

Global_active_power      25979
Global_reactive_power    25979
Voltage                  25979
Global_intensity         25979
Sub_metering_1           25979
Sub_metering_2           25979
Sub_metering_3               0
dtype: int64

In [16]:
df.replace('?', 'NaN', inplace=True)

In [17]:
df['Global_active_power'] = df['Global_active_power'].astype('float')
df['Global_reactive_power'] = df['Global_reactive_power'].astype('float')
df['Voltage'] = df['Voltage'].astype('float')
df['Global_intensity'] = df['Global_intensity'].astype('float')
df['Sub_metering_1'] = df['Sub_metering_1'].astype('float')
df['Sub_metering_2'] = df['Sub_metering_2'].astype('float')

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2075259 entries, 2006-12-16 17:24:00 to 2010-11-26 21:02:00
Data columns (total 7 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   Global_active_power    float64
 1   Global_reactive_power  float64
 2   Voltage                float64
 3   Global_intensity       float64
 4   Sub_metering_1         float64
 5   Sub_metering_2         float64
 6   Sub_metering_3         float64
dtypes: float64(7)
memory usage: 126.7 MB


In [19]:
df.describe(include='all').T

,count,mean,std,min,25%,50%,75%,max
Global_active_power,2049280.0,1.091615,1.057294,0.076,0.308,0.602,1.528,11.122
Global_reactive_power,2049280.0,0.123714,0.112722,0.000,0.048,0.100,0.194,1.390
Voltage,2049280.0,240.839858,3.239987,223.200,238.990,241.010,242.890,254.150
Global_intensity,2049280.0,4.627759,4.444396,0.200,1.400,2.600,6.400,48.400
Sub_metering_1,2049280.0,1.121923,6.153031,0.000,0.000,0.000,0.000,88.000
Sub_metering_2,2049280.0,1.298520,5.822026,0.000,0.000,0.000,1.000,80.000
Sub_metering_3,2049280.0,6.458447,8.437154,0.000,0.000,1.000,17.000,31.000


In [ ]:
# Fill missing values with a value at the same time, one day ago

In [20]:
df.isna().sum()

Global_active_power      25979
Global_reactive_power    25979
Voltage                  25979
Global_intensity         25979
Sub_metering_1           25979
Sub_metering_2           25979
Sub_metering_3           25979
dtype: int64

In [21]:
df = df.fillna(df.shift(60*24))

In [22]:
df.isna().sum()

Global_active_power      16007
Global_reactive_power    16007
Voltage                  16007
Global_intensity         16007
Sub_metering_1           16007
Sub_metering_2           16007
Sub_metering_3           16007
dtype: int64

In [23]:
df = df.fillna(df.shift(60*24))

In [24]:
df.isna().sum()

Global_active_power      8220
Global_reactive_power    8220
Voltage                  8220
Global_intensity         8220
Sub_metering_1           8220
Sub_metering_2           8220
Sub_metering_3           8220
dtype: int64

In [25]:
df = df.fillna(df.shift(60*24))

In [26]:
df.isna().sum()

Global_active_power      3823
Global_reactive_power    3823
Voltage                  3823
Global_intensity         3823
Sub_metering_1           3823
Sub_metering_2           3823
Sub_metering_3           3823
dtype: int64

In [27]:
df = df.fillna(df.shift(60*24))

In [28]:
df.isna().sum()

Global_active_power      1466
Global_reactive_power    1466
Voltage                  1466
Global_intensity         1466
Sub_metering_1           1466
Sub_metering_2           1466
Sub_metering_3           1466
dtype: int64

In [29]:
df = df.fillna(df.shift(60*24))

In [30]:
df.isna().sum()

Global_active_power      26
Global_reactive_power    26
Voltage                  26
Global_intensity         26
Sub_metering_1           26
Sub_metering_2           26
Sub_metering_3           26
dtype: int64

In [31]:
df = df.fillna(df.shift(60*24))

In [32]:
df.isna().sum()

Global_active_power      0
Global_reactive_power    0
Voltage                  0
Global_intensity         0
Sub_metering_1           0
Sub_metering_2           0
Sub_metering_3           0
dtype: int64

In [34]:
df.describe(include='all').T

,count,mean,std,min,25%,50%,75%,max
Global_active_power,2075259.0,1.089418,1.054678,0.076,0.308,0.602,1.526,11.122
Global_reactive_power,2075259.0,0.123687,0.112593,0.000,0.048,0.100,0.194,1.390
Voltage,2075259.0,240.836427,3.240051,223.200,238.990,241.000,242.870,254.150
Global_intensity,2075259.0,4.618401,4.433165,0.200,1.400,2.600,6.400,48.400
Sub_metering_1,2075259.0,1.118474,6.141460,0.000,0.000,0.000,0.000,88.000
Sub_metering_2,2075259.0,1.291131,5.796922,0.000,0.000,0.000,1.000,80.000
Sub_metering_3,2075259.0,6.448635,8.433584,0.000,0.000,1.000,17.000,31.000


In [35]:
df

,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
DateTime,,,,,,,
2006-12-16 17:24:00,4.216,0.418,234.84,18.4,0.0,1.0,17.0
2006-12-16 17:25:00,5.360,0.436,233.63,23.0,0.0,1.0,16.0
2006-12-16 17:26:00,5.374,0.498,233.29,23.0,0.0,2.0,17.0
2006-12-16 17:27:00,5.388,0.502,233.74,23.0,0.0,1.0,17.0
2006-12-16 17:28:00,3.666,0.528,235.68,15.8,0.0,1.0,17.0
...,...,...,...,...,...,...,...
2010-11-26 20:58:00,0.946,0.000,240.43,4.0,0.0,0.0,0.0
2010-11-26 20:59:00,0.944,0.000,240.00,4.0,0.0,0.0,0.0
2010-11-26 21:00:00,0.938,0.000,239.82,3.8,0.0,0.0,0.0
